In [ ]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import whois
import datetime
import time
import httplib2
import urllib
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
df = pd.read_csv(r'C:\Users\Matt\Downloads\URL_data.csv')

In [ ]:
#https in domain name section
df['https_in_domain'] = df.loc['/https'] = 1.0 if '/https' in df['url'] else 0.0
#may be better applying this to just the domain name section if we split on that
#and then taking the / out of '/https'

In [ ]:
#Gets full html code of the webpage and stores to new column, html_code
for row_idx, row in df.iterrows():
    try:
        page = urllib.request.urlopen(row['url'])
        soup = BeautifulSoup(page)
        df.loc[row_idx, 'html_code'] = str(soup)
    except:
        df.loc[row_idx, 'html_code'] = "Error"

In [ ]:
# Gets <a> tags from html_code column and print to new column, anchors
for row_idx, row in df.iterrows():
    try:
        soup = BeautifulSoup(row['html_code'])
        df.loc[row_idx, 'anchors'] = str(soup.a)
    except:
        df.loc[row_idx, 'anchors'] = "Error"
# Need to compare with domain and check for other discrepencies instead

In [ ]:
# Gets <form> tags from html_code column and print to new column, forms
for row_idx, row in df.iterrows():
    try:
        soup = BeautifulSoup(row['html_code'])
        df.loc[row_idx, 'forms'] = str(soup.find_all("form"))
    except:
        df.loc[row_idx, 'forms'] = "Error"

In [ ]:
# SFH has about:blank in form handler
df['SFH_about:blank'] = df.loc['about:blank'] = 1.0 if 'about:blank' in df['forms'] else 0.0

In [ ]:
# Tries to submit to email in html code
sus_lis = ['mail()','mailto:']
for i in sus_list:
    df['submit_to_email'] = df.loc[i]= 1.0 if i in df['html_code'] else 0.0
#sub['submit_to_email2'] = sub.loc['mailto:'] = 1.0 if 'mailto:' in sub['html_code'] else 0.0

In [ ]:
# Check number of redirects
for row_idx, row in df.iterrows():
    try:
        r = requests.get(row['url'])
        df.loc[row_idx, 'redirects'] = r.history
        if (len(r.history)) > 1:
            df.loc[row_idx, 'redirect_suspicious'] = 1
        else:
            df.loc[row_idx, 'redirect_suspicious'] = 0
    except:
        df.loc[row_idx, 'redirects'] = "Error"

In [ ]:
# Check for double click being disabled
df['DoubleClickDisabled'] = df.loc['event.button==2'] = 1.0 if 'event.button==2' in df['html_code'] else 0.0

In [ ]:
# Check for onMouseOver
for row_idx, row in df.iterrows():
    try:
        soup = BeautifulSoup(row['html_code'])
        df.loc[row_idx, 'MouseOver'] = str(soup.find_all(onmouseover=True))
    except:
        df.loc[row_idx, 'MouseOver'] = "None Found/Error"

In [ ]:
# Get favicon link 
for row_idx, row in sub.iterrows():
    try:
        page = urllib.request.urlopen(row['url'])
        soup = BeautifulSoup(page)
        icon_link = soup.find("link", rel="shortcut icon")
        sub.loc[row_idx, 'fav_link'] = str(icon_link)
    except:
        sub.loc[row_idx, 'fav_link'] = "Error"

In [ ]:
# Write to EXCEL file
df.to_excel('***path.xlsx')